# Ratterdam Stimulus Signal Extraction Given Non-Stationarity of Data

Purpose of this notebook is to develop and test approaches to pull out a texture encoding signal (e.g via decoding) in a way that explicitly  
accounts for and minimizes theproblem of the temporal non-stationarity we see in the Ratterdam data.

WH 20200611

Table of Contents:
1. Windowed Bayesian Classifier

## Windowed Bayesian CLassifier

In [1]:
# Imports 
import numpy as np, itertools, scipy, sys, os, matplotlib.pyplot as plt
sys.path.insert(0, 'E:\\UserData\\Documents\\GitHub\\ratterdam\\')

import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import ratterdam_Defaults as Def
import ratterdam_DataFiltering as Filt
import utility_fx as util

In [2]:
%qtconsole --style native
%matplotlib qt5

In [3]:
rat = "R808"
expCode = "BRD6"
dayPath = f"E:\\Ratterdam\\{rat}\\{rat}{expCode}\\"
clustName = "TT15\\cl-maze1.2"
unit = util.loadUnit(dayPath, expCode, clustName)

E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:171: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:171: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:177: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:177: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:326: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


In [27]:
# Single unit data collection and MLE calculation for each feature, specified in a list below
features = ['com', 'comVal', 'AUC', 'max']
# For a single alley, collect all trials regardless of texture. 
alley = Def.beltwayAlleys[0]
rms = np.empty((0, Def.singleAlleyBins[0]-1))
realIdx = np.empty((0)) # trial number overall i.e. if rewards were included (which they should not be)
for visit in unit.alleys[alley]:
    rms = np.vstack((rms, visit['ratemap1d']))
    realIdx = np.hstack((realIdx, visit['metadata']['nrtrialnum']))
    
# Define a window size, a step size, and a function which takes a set of trials and computes the MLE of a set of features
# Each feature has its own function to calc its respective MLE and this function will be based to the function which takes the set of trials
windowSize = 10 # 10 or so should be minimum so there are at least a couple of samples of each stim on average
stepSize = 2 # shift amount in number of trials of the sliding window. Optimal value unclear 20200611

def computeMLE_FeatureVector(data, features):
    """
    Data is an (n,b) array of n ratemaps each
    consisting of firing rate in b spatial bins
    Features is a list of features, each of which
    will have its MLE computed over the sample
    """
    